#### Overlap testing

In order to address the problems with double-counting, I need to remove the areas of overlap between Natura 2000 sites. In this file, I perform some testing to make sure I remove the overlaps correctly.

My plan for removing the overlaps is basically to dissolve all the polygon boundaries to create to make one big polygon with no overlaps. I then 'explode' the geometry so that the massive multi-part geomtry is separate into individual polygons for each part. This step is needed because the big dissolved multi-part polygon is too big to calculate zonal statistics.

My worry with this approach is that because I would end up creating smaller polygons than in the original dataset (the original dataset includes some multi-part Natura sites), this may mean the area gets calculated differently. In theory I think it should not matter, but I would like to check.

To check for this issue, I do the following steps:
1) Create a test site which is a multi-part polygon with some areas/parts that are very small (small enough not to return results) -- I do this part manually in QGIS
2) Calculate the zonal statistics for this test site in it's original form
3) Separate ('explode') the test site into individual polygons
4) Calculate the zonal statistics for the separated/exploded geometries (and sum them up)
5) Compare the two sets of zonal statistics

My hope is that these would return the same results, even though the second exploded geometry set might return some empty results for the smaller polygons. I think these would also be missing from the result for the multi-part geometry set, but it just wouldn't be clear from the results because you only get a single result back for the whole multi-part polygon. If the results match then I think the dissolve/explode approach to remove the overlaps is a good way forward. 

In [1]:
# SETUP

from rasterstats import zonal_stats
import geopandas as gpd

# Because I'm working in the 'other' folder, I need to store the main directory info
main_dir = "C:/Users/ninam/Documents/UZH/04_Thesis/code/nm_forest_thesis/"

In [9]:
# CALC MULTIPART STATS

# Load test site (multi-part geom with 1 main part and 2 small parts)
test_site = gpd.read_file(main_dir + "processing/test_site_multipart.shp")

# Load in ESA FNF (for testing)
esa_FNF = main_dir + "outputs/esa_lccs_class_3035_DE_5m_2018_FNF.tif"

# Calculate the zonal stats (count only - just need the total area)
test_site_multipart_stats = zonal_stats(test_site, esa_FNF, stats=['count'],
                                        geojson_out=True)

In [ ]:
# FORMAT RESULTS

# Load as feature collection
test_site_multipart_stats_fc = {"type": "FeatureCollection","features": test_site_multipart_stats}

# Extract features
test_site_multipart_stats_features = test_site_multipart_stats_fc["features"]

# Check results
#test_site_multipart_stats_features

In [12]:
# EXTRACT & CALC TOTAL AREA (MULTIPART)

# Create empty lists for storing the area counts
multipart_count_list = []

# Fill the list with all the area counts
for row in test_site_multipart_stats_features:
    multipart_count_list.append(row['properties']['count'])

# Sum all the values and convert to area (25m2 pixel size = 0.0025 ha)
multipart_total_area_ha = sum(multipart_count_list) * 0.0025

# Show the result
print("Total area (ha) for single multipart geom = ", multipart_total_area_ha)

Total area (ha) for single multipart geom =  22.915


In [13]:
# CREATE EXPLODED VERSION & CALC STATS

# Explode the multipart
test_site_exp = test_site.explode()

# Calculate the zonal stats (count only - just need the total area)
test_site_exp_stats = zonal_stats(test_site_exp, esa_FNF, stats=['count'],
                                  geojson_out=True)

In [15]:
# FORMAT RESULTS

# Load as feature collection
test_site_exp_stats_fc = {"type": "FeatureCollection","features": test_site_exp_stats}

# Extract features
test_site_exp_stats_features = test_site_exp_stats_fc["features"]

# Check results
test_site_exp_stats_features

[{'id': '0',
  'type': 'Feature',
  'properties': {'id': 1, 'count': 0},
  'geometry': {'type': 'Polygon',
   'coordinates': (((4415740.052809142, 3132526.347389419),
     (4415741.647363373, 3132525.721678265),
     (4415741.647363373, 3132525.721678265),
     (4415742.515285296, 3132523.8041763413),
     (4415741.869389911, 3132522.1894378792),
     (4415739.972072219, 3132521.3618844175),
     (4415738.357333756, 3132522.007779802),
     (4415737.549964526, 3132523.8647290333),
     (4415738.074754526, 3132525.4794674953),
     (4415740.052809142, 3132526.347389419)),)},
  'bbox': (4415737.549964526,
   3132521.3618844175,
   4415742.515285296,
   3132526.347389419)},
 {'id': '0',
  'type': 'Feature',
  'properties': {'id': 1, 'count': 4},
  'geometry': {'type': 'Polygon',
   'coordinates': (((4431109.052754132, 3131860.590721509),
     (4431119.104501058, 3131860.954037663),
     (4431119.306343365, 3131850.98302766),
     (4431109.294964901, 3131850.660079967),
     (4431109.05275

The printout above shows that the first geometry does not return any results (count = 0) but the other geoms do.

In [16]:
# EXTRACT & CALC TOTAL AREA (MULTIPART)

# Create empty list for storing the area counts
exp_count_list = []

# Fill the list with all the area counts
for row in test_site_exp_stats_features:
    exp_count_list.append(row['properties']['count'])

# Sum all the values and convert to area (25m2 pixel size = 0.0025 ha)
exploded_total_area_ha = sum(exp_count_list) * 0.0025

# Show the result
print("Total area (ha) for exploded geoms = ", exploded_total_area_ha)

Total area (ha) for exploded geoms =  22.915


The results are exactly the same - as hoped/expected. So the by dissolve the geometries I can get rid of the overlaps, and by exploding to single geometries I can get the zonal statistics to run without affecting the results. 

#### One additional test:

1) Calculate the zonal stats on the esa fnf map using the dissolve/explode approach for the Natura sites and sum up the count stats (to get the area of analysis)
2) In QGIS, calculate dissolve the Natura sites (but don't explode) and clip the sites to the CORINE footprint to get the approximate same area of analysis from step 1. Calculate the total area of this dissolved/clipped area
3) Compare results. They won't be exactly the same as in step 1 it's based of the pixel counts of the underlying raster, but in step 2 it's based on an area calculation from the vector... but I think they should be similar. 

In [2]:
# CREATE THE DISSOLVE/EXPLODE GEOMS

# Load Natura shp
natura = gpd.read_file(main_dir + "outputs/natura2000_3035_DE.shp", 
                       columns=["SITECODE", "SITENAME", "MS", "SITETYPE"])

# Dissolve all geometries to make one big polygon (with no overlaps)
big_dissolved = natura.dissolve()

# Explore multi-part polygon into individual polygons
# This is needed because the big_dissolved polygon is too big to calc zonal stats
natura_wo_overlap = big_dissolved.explode()

# Save non-overlapping geoms to shp
natura_wo_overlap.to_file(main_dir + "processing/natura_wo_overlap_test.shp")

In [5]:
# CALC ZONAL STATS FOR DISSOLVE/EXPLODE GEOMS

# Load in dissolved/exploded geoms
natura_wo_overlap = gpd.read_file(main_dir + "processing/natura_wo_overlap_test.shp")

# Load in ESA FNF (for testing)
esa_FNF = main_dir + "outputs/esa_lccs_class_3035_DE_5m_2018_FNF.tif"

# Calculate the zonal stats (count only - just need the total area)
esaFNF_natura_wo_overlaps_stats = zonal_stats(natura_wo_overlap, esa_FNF,
                                              stats=['count'],
                                              geojson_out=True)

In [ ]:
# FORMAT RESULTS

# Load as feature collection
esaFNF_natura_wo_overlaps_stats_fc = {"type": "FeatureCollection","features": esaFNF_natura_wo_overlaps_stats}

# Extract features
esa_wo_overlap_features = esaFNF_natura_wo_overlaps_stats_fc["features"]

# Check results
#esa_wo_overlap_features

In [8]:
# EXTRACT & CALC TOTAL AREA

# Create empty lists for storing the area counts
count_list = []

# Fill the list with all the area counts
for row in esa_wo_overlap_features:
    count_list.append(row['properties']['count'])

# Sum all the values and convert to area (25m2 pixel size = 0.0025 ha)
esa_wo_overlap_total_area_ha = sum(count_list) * 0.0025

# Show the result
print("Total area (ha) for exploded/dissolved geoms = ", esa_wo_overlap_total_area_ha)

Total area (ha) for exploded/dissolved geoms =  5519331.68


The results for the dissolved (but not exploded) test in QGIS was a total area of 55193.367 km2 or about 5.5 million ha. The two results are almost identical! This seems to confirm that dissolving and exploding achieves the same outcome as just dissolving - in other words, the exploding part doesn't effect the results by making the geometries too small. 

This area corresponds to the total area assessed for this thesis **without counting overlaps**.